# TCG Shadowbox Generator - Quick Start

このノートブックは **ALL RUN** で一気に実行できるシンプルなクイックスタートです。

**使い方:**
1. 下の「設定セクション」で変数を設定
2. メニューから **Run > Run All Cells** を実行
3. 結果を確認

> 詳細な実験や各検出メソッドの比較は `02_detailed_experiments.ipynb` を参照してください。

---
## 設定セクション

以下の変数を設定してください。

In [ ]:
# =============================================================================
# 設定変数（ここを編集してください）
# =============================================================================

# --- 画像ソース ---
# 画像の読み込み方法を選択: "url", "file", "directory"
IMAGE_MODE = "url"

# URL指定の場合
IMAGE_URL = "https://example.com/your-card-image.png"

# ファイルパス指定の場合
IMAGE_FILE = "../data/cards/your-card.png"

# ディレクトリ指定の場合（ギャラリーから選択）
IMAGE_DIR = "../data/cards/"
IMAGE_INDEX = 0  # 選択する画像のインデックス（0から始まる）

# --- イラスト領域検出 ---
# 検出方法: "auto", "manual", または特定のメソッド名
# 利用可能なメソッド:
#   "auto"                    - 自動（最も信頼度の高い結果を使用）
#   "gradient_richness"       - 勾配の豊かさ（おすすめ）
#   "edge_detection"          - エッジ検出
#   "hsv_threshold"           - HSV閾値
#   "contour_detection"       - 輪郭検出
#   "grid_scoring"            - グリッドスコアリング
#   "boundary_contrast"       - 境界コントラスト
#   "frame_analysis"          - フレーム解析
#   "band_complexity"         - 水平帯複雑度
#   "horizontal_lines"        - 水平線検出
#   "center_expansion"        - 中央拡張
#   "manual"                  - 手動指定（MANUAL_BBOX を設定）
DETECTION_METHOD = "auto"

# 手動指定の場合のバウンディングボックス (x, y, width, height)
MANUAL_BBOX = (50, 100, 400, 300)

# --- 処理モード ---
# True: 生深度モード（クラスタリングなし、滑らかな深度）
# False: クラスタリングモード（離散的なレイヤー、シャドーボックス感強い）
USE_RAW_DEPTH = False

# クラスタリングモード時のレイヤー数（Noneで自動探索）
NUM_LAYERS = 5

# 生深度モード時の深度スケール（大きいほど立体感が増す）
DEPTH_SCALE = 1.5

# --- 表示オプション ---
# カードフレームを含めるか（True: 本物のシャドーボックス風）
INCLUDE_CARD_FRAME = True

# 3Dフレーム（額縁）を表示するか
SHOW_FRAME = True

# --- モックモード（テスト用） ---
# True: モック深度推定（高速、モデル不要）
# False: 実際の深度推定（初回はモデルダウンロードあり）
USE_MOCK_DEPTH = False

print("設定完了!")

---
## セットアップ

In [ ]:
# インポート
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from shadowbox import create_pipeline
from shadowbox.config import BoundingBox, ShadowboxSettings
from shadowbox.detection import RegionDetector, detect_illustration_region
from shadowbox.gui import ImageSelector, load_from_url
from shadowbox.utils.image import load_image
from shadowbox.visualization import (
    create_depth_heatmap,
    show_clustering_summary,
    render_shadowbox,
    RenderOptions,
)

# matplotlibの日本語フォント設定（Windows）
import matplotlib
matplotlib.rcParams['font.family'] = ['MS Gothic', 'Yu Gothic', 'Meiryo', 'sans-serif']
matplotlib.rcParams['axes.unicode_minus'] = False

print("セットアップ完了!")

---
## 1. 画像読み込み

In [ ]:
# 画像を読み込み
if IMAGE_MODE == "url":
    print(f"URLから読み込み: {IMAGE_URL}")
    image = load_from_url(IMAGE_URL)
elif IMAGE_MODE == "file":
    print(f"ファイルから読み込み: {IMAGE_FILE}")
    image = load_image(IMAGE_FILE)
elif IMAGE_MODE == "directory":
    print(f"ディレクトリから読み込み: {IMAGE_DIR}")
    selector = ImageSelector(IMAGE_DIR)
    selector.show_gallery()  # ギャラリー表示
    image = selector.select(IMAGE_INDEX)
    print(f"選択: [{IMAGE_INDEX}] {selector.images[IMAGE_INDEX].name}")
else:
    raise ValueError(f"未知のIMAGE_MODE: {IMAGE_MODE}")

print(f"画像サイズ: {image.size[0]} x {image.size[1]}")

# 画像を表示
plt.figure(figsize=(6, 8))
plt.imshow(image)
plt.title("入力画像")
plt.axis("off")
plt.show()

---
## 2. イラスト領域検出

In [ ]:
# イラスト領域を検出
if DETECTION_METHOD == "manual":
    # 手動指定
    bbox = BoundingBox(
        x=MANUAL_BBOX[0],
        y=MANUAL_BBOX[1],
        width=MANUAL_BBOX[2],
        height=MANUAL_BBOX[3],
    )
    detection_method = "manual"
    confidence = 1.0
elif DETECTION_METHOD == "auto":
    # 自動検出（最良の結果を使用）
    result = detect_illustration_region(image)
    bbox = result.bbox
    detection_method = result.method
    confidence = result.confidence
else:
    # 特定のメソッドを使用
    detector = RegionDetector(min_area_ratio=0.05, max_area_ratio=0.95)
    cv_image = detector._pil_to_cv(image)
    method_map = {
        "gradient_richness": detector._detect_by_gradient_richness,
        "edge_detection": detector._detect_by_edges,
        "hsv_threshold": detector._detect_by_hsv_threshold,
        "contour_detection": detector._detect_by_contours,
        "grid_scoring": detector._detect_by_grid_scoring,
        "boundary_contrast": detector._detect_by_boundary_contrast,
        "frame_analysis": detector._detect_by_frame_analysis,
        "band_complexity": detector._detect_by_band_complexity,
        "horizontal_lines": detector._detect_by_horizontal_lines,
        "center_expansion": detector._detect_by_center_expansion,
    }
    if DETECTION_METHOD in method_map:
        result = method_map[DETECTION_METHOD](cv_image)
        if result:
            bbox = result.bbox
            detection_method = result.method
            confidence = result.confidence
        else:
            print(f"警告: {DETECTION_METHOD} では検出できませんでした。自動検出にフォールバック。")
            result = detect_illustration_region(image)
            bbox = result.bbox
            detection_method = result.method
            confidence = result.confidence
    else:
        print(f"警告: 未知のメソッド '{DETECTION_METHOD}'。自動検出を使用。")
        result = detect_illustration_region(image)
        bbox = result.bbox
        detection_method = result.method
        confidence = result.confidence

print(f"検出方法: {detection_method}")
print(f"信頼度: {confidence:.2f}")
print(f"領域: x={bbox.x}, y={bbox.y}, w={bbox.width}, h={bbox.height}")

# 検出結果を可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# 元画像と検出領域
axes[0].imshow(image)
rect = plt.Rectangle(
    (bbox.x, bbox.y), bbox.width, bbox.height,
    linewidth=3, edgecolor='lime', facecolor='none'
)
axes[0].add_patch(rect)
axes[0].set_title(f"検出結果: {detection_method}\n信頼度: {confidence:.2f}")
axes[0].axis("off")

# クロップ領域プレビュー
cropped_preview = image.crop((bbox.x, bbox.y, bbox.x + bbox.width, bbox.y + bbox.height))
axes[1].imshow(cropped_preview)
axes[1].set_title(f"イラスト領域\n({bbox.width} x {bbox.height} px)")
axes[1].axis("off")

plt.tight_layout()
plt.show()

---
## 3. 深度推定 & メッシュ生成

In [ ]:
# パイプラインを作成
pipeline = create_pipeline(use_mock_depth=USE_MOCK_DEPTH)

# 処理を実行
result = pipeline.process(
    image,
    custom_bbox=bbox,
    k=NUM_LAYERS,
    include_frame=SHOW_FRAME,
    include_card_frame=INCLUDE_CARD_FRAME,
    use_raw_depth=USE_RAW_DEPTH,
    depth_scale=DEPTH_SCALE,
)

mode_name = "生深度モード" if USE_RAW_DEPTH else f"クラスタリングモード (k={result.optimal_k})"
print(f"処理モード: {mode_name}")
print(f"レイヤー数: {result.mesh.num_layers}")
print(f"総頂点数: {result.mesh.total_vertices:,}")
print(f"深度範囲: {result.depth_map.min():.3f} ~ {result.depth_map.max():.3f}")

---
## 4. 2D可視化（深度ヒートマップ & クラスタリングサマリー）

In [ ]:
# 深度ヒートマップ
fig, axes = create_depth_heatmap(
    result.depth_map,
    result.cropped_image,
    cmap="viridis",
    title="深度推定結果",
)
plt.show()

In [ ]:
# クラスタリングサマリー（クラスタリングモードの場合のみ意味がある）
if not USE_RAW_DEPTH:
    fig, axes = show_clustering_summary(
        result.cropped_image,
        result.depth_map,
        result.labels,
        result.centroids,
    )
    plt.show()
else:
    print("生深度モードではクラスタリングサマリーはスキップされます。")

---
## 5. 3Dシャドーボックス表示

**操作方法:**
- マウス左ドラッグ: 回転
- マウス右ドラッグ: パン
- スクロール: ズーム

In [ ]:
# レンダリングオプション
options = RenderOptions(
    background_color=(30, 30, 40),
    point_size=4.0,
    show_frame=SHOW_FRAME,
    window_size=(1000, 800),
    title="TCG Shadowbox Viewer",
)

# 3Dレンダリング
render_shadowbox(result.mesh, options)

---
## 次のステップ

- **設定を変更して再実行**: 上の設定セクションを編集し、「Run All」で再実行
- **詳細な実験**: `02_detailed_experiments.ipynb` で各検出メソッドの比較や手動選択を試す
- **パラメータ調整**:
  - `NUM_LAYERS`: レイヤー数を増減してシャドーボックスの段数を調整
  - `USE_RAW_DEPTH`: True で滑らかな深度表現、False で段差のあるシャドーボックス風
  - `DEPTH_SCALE`: 生深度モードで立体感を調整
  - `INCLUDE_CARD_FRAME`: カード全体を含めた本物のシャドーボックス風表示